In [ ]:
!pip install google-play-scraper --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.5 MB/s eta 0:00:00


In [1]:
from google_play_scraper import Sort, reviews
import pandas as pd



In [2]:
def fetch_reviews(app_id, lang, country='ae', total_count=1000):
    print(f"\n🌐 بدء سحب التعليقات للغة: {lang} ...")
    all_reviews = []
    next_token = None
    round_num = 1

    while len(all_reviews) < total_count:
        print(f"🔄 الجولة {round_num}: جاري جلب دفعة جديدة من التعليقات...")

        reviews_batch, next_token = reviews(
            app_id,
            lang=lang,
            country=country,
            sort=Sort.NEWEST,
            count=200,
            continuation_token=next_token
        )

        all_reviews.extend(reviews_batch)
        print(f"✅ تم جلب {len(reviews_batch)} تعليق (الإجمالي حتى الآن: {len(all_reviews)})")

        # إذا انتهت التعليقات أو ما رجع دفعة كاملة
        if not next_token or len(reviews_batch) < 200:
            print("🚫 لا توجد دفعات إضافية.")
            break

        round_num += 1

        # أمان إضافي: وقف بعد 10 جولات
        if round_num > 10:
            print("⚠️ تم الوصول إلى الحد الأقصى للجولات (10). إيقاف مؤقت.")
            break

    print(f"🎯 انتهى السحب للغة {lang}. مجموع التعليقات: {len(all_reviews)}\n")

    df = pd.DataFrame(all_reviews)[['userName', 'content', 'score']]
    df.rename(columns={
        'userName': 'Name',
        'content': 'Comment',
        'score': 'Rating'
    }, inplace=True)
    df['Language'] = lang
    return df

# ✅ المعرف الرسمي لتطبيق RAKBANK
app_id = "com.rak"

print("🚀 بدء استخراج التعليقات من Google Play لتطبيق RAKBANK...")

df_ar = fetch_reviews(app_id, lang='ar', total_count=5000)
df_en = fetch_reviews(app_id, lang='en', total_count=5000)



🚀 بدء استخراج التعليقات من Google Play لتطبيق RAKBANK...

🌐 بدء سحب التعليقات للغة: ar ...
🔄 الجولة 1: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 200)
🔄 الجولة 2: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 61 تعليق (الإجمالي حتى الآن: 261)
🚫 لا توجد دفعات إضافية.
🎯 انتهى السحب للغة ar. مجموع التعليقات: 261


🌐 بدء سحب التعليقات للغة: en ...
🔄 الجولة 1: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 200)
🔄 الجولة 2: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 400)
🔄 الجولة 3: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 600)
🔄 الجولة 4: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 800)
🔄 الجولة 5: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 1000)
🔄 الجولة 6: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (الإجمالي حتى الآن: 1200)
🔄 الجولة 7: جاري جلب دفعة جديدة من التعليقات...
✅ تم جلب 200 تعليق (ا

In [3]:
print("🧩 دمج نتائج اللغتين...")
df_all = pd.concat([df_ar, df_en], ignore_index=True)


🧩 دمج نتائج اللغتين...


In [17]:
import pandas as pd
from IPython.display import display

rows = []

for rating in range(1, 6):
    subset = df_all[df_all['Rating'] == rating]
    count = len(subset)

    if count > 0:
        sample = subset.sample(1).iloc[0]
        rows.append({
            'Rating': rating,
            'Count': count,
            'Sample_Name': sample['Name'],
            'Sample_Comment': sample['Comment']
        })
    else:
        rows.append({
            'Rating': rating,
            'Count': 0,
            'Sample_Name': 'لا يوجد',
            'Sample_Comment': 'لا يوجد تعليقات بهذا التقييم'
        })

# إنشاء DataFrame من النتائج
summary_df = pd.DataFrame(rows)

# عرض النتائج
display(summary_df)


,Rating,Count,Sample_Name,Sample_Comment
0,1,754,مستخدم Google,بدون اللغة العربيه ما يسوى شي
1,2,111,мαlιк. innocent,zero balance account?
2,3,114,مستخدم Google,لزم الغة العربية
3,4,157,Selvakumar Annadurai,Good
4,5,1125,Anil At,very good


In [6]:
df_all

,Name,Comment,Rating,Language
0,Obaid Eissa,كل البنوك تحط الهويه الرقميه ياليت يصير تطوير ...,1,ar
1,Business Bin Hareb Center,ok,5,ar
2,MOHAMMED ALZAABI,برنامج ممتاز بمعني الكلمه وخاصة بعد اضافة اللغ...,5,ar
3,Ziko max,من أسوأ البنوك فى الامارات,1,ar
4,khalid tarabieh,سهل و رائع,5,ar
...,...,...,...,...
2256,pramodan puthiyedath,Excellent,5,en
2257,Jagtar singh Jagga,Good service,4,en
2258,Imran Alam,After last update its not working for full acc...,3,en
2259,Jibu Thomas,Nice,5,en


In [4]:
output_file = "rakbank_reviews_all.csv"
df_all.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"💾 تم حفظ جميع التعليقات في الملف: {output_file}")


💾 تم حفظ جميع التعليقات في الملف: rakbank_reviews_all.csv


In [26]:
target_name = "بهاء كريدي"

user_reviews = df_all[df_all['Name'].str.strip().str.upper() == target_name.upper()]

# عرض النتائج
if user_reviews.empty:
    print(f"❌ لا يوجد تعليق باسم: {target_name}")
else:
    print(f"✅ تم العثور على {len(user_reviews)} تعليق لهذا المستخدم:")
    print(user_reviews[['Name', 'Comment', 'Rating', 'Language']])

✅ تم العثور على 1 تعليق لهذا المستخدم:
           Name                                            Comment  Rating  \
958  بهاء كريدي  البنك الأسوأ، خدماتهم بطيئة، التطبيق غالبا لا ...       1   

    Language  
958       en  


In [8]:
from google_play_scraper import app

app_id = "com.rak"  # معرّف تطبيق RAKBANK

app_info = app(app_id, lang='en', country='ae')

# عرض عدد المراجعات
print(f"📊 عدد التقييمات الكلي (Ratings): {app_info['ratings']:,}")
print(f"📝 عدد التعليقات النصية المتوفرة حاليًا (Reviews): {app_info['reviews']:,}")


📊 عدد التقييمات الكلي (Ratings): 33,869
📝 عدد التعليقات النصية المتوفرة حاليًا (Reviews): 2,581
